In [1]:
import os 
try:
    import earthpy as et
except:
    !pip install --user earthpy
    import earthpy as et
try:    
    import folium
except:
    !pip install --user folium
    import folium
    
from folium import plugins

try:
    import rasterio as rio
except:
    !pip install --user rasterio
    import rasterio as rio
    
from rasterio.warp import calculate_default_transform, reproject, Resampling

# Set working directory to earth-analytics
!mkdir ~/earth-analytics
os.chdir(os.path.join(et.io.HOME, 'earth-analytics'))


mkdir: cannot create directory ‘/home/dario/earth-analytics’: File exists


In [2]:
import pandas as pd
import numpy as np
''' 
# Make a data frame with dots to show on the map
data = pd.DataFrame({
   'lat':[-58, 2, 145, 30.32, -4.03, -73.57, 36.82, -38.5],
   'lon':[-34, 49, -38, 59.93, 5.33, 45.52, -1.29, -12.97],
   'name':['Buenos Aires', 'Paris', 'melbourne', 'St Petersbourg', 'Abidjan', 'Montreal', 'Nairobi', 'Salvador'],
   'value':[10,12,40,70,23,43,100,43]
})
data
'''

data=pd.read_csv("/home/dario/NeoMeetup/Visualizations/map/member_event_dist_from_milan.csv")
data.head()
data.sort_values("event_id")

,member_id,event_id,city,lat_member,lon_member,lat_event,lon_event,distanza
653,266518837,255828197,Milano,45.46,9.19,45.46,9.19,0.000000
652,262172301,255828197,Milano,45.46,9.19,45.46,9.19,0.000000
595,225336599,257680863,Milano,45.46,9.19,45.46,9.19,0.000000
596,12767885,257680863,Milano,45.46,9.19,45.46,9.19,0.000000
597,277940591,257680863,Milano,45.46,9.19,45.46,9.19,0.000000
1274,246061953,258124335,Milano,45.46,9.19,45.46,9.19,0.000000
246,275215287,258124335,Milano,45.55,9.30,45.46,9.19,13.191883
1275,276912135,258124335,Milano,45.46,9.19,45.46,9.19,0.000000
1276,274045959,258124335,Milano,45.46,9.19,45.46,9.19,0.000000
315,274826062,258842222,Milano,45.49,9.16,45.46,9.19,4.078823


In [3]:
for line in data.itertuples():
    if data.iloc[line.Index]['distanza']==0:
        data['distanza'].at[line.Index]=7
data        

,member_id,event_id,city,lat_member,lon_member,lat_event,lon_event,distanza
0,131238972,260314253,Milano,51.47,-0.62,45.46,9.19,984.326958
1,255249321,260510292,Milano,53.43,14.53,45.46,9.19,967.078594
2,193264733,260048808,Milano,51.54,-0.19,45.46,9.19,966.453812
3,220047390,259876979,Milano,51.57,-0.11,45.46,9.19,964.464020
4,253141302,260001995,Milano,51.51,-0.14,45.46,9.19,961.650265
5,266273412,260306056,Milano,51.52,-0.10,45.46,9.19,960.284099
6,235505266,260088430,Milano,51.52,-0.10,45.46,9.19,960.284099
7,246078536,259124091,Milano,51.50,-0.08,45.46,9.19,957.778125
8,246078536,259530309,Milano,51.50,-0.08,45.46,9.19,957.778125
9,247779838,sdgwsqyzgbgb,Milano,51.51,-0.04,45.46,9.19,956.418862


In [4]:
data.sort_values("distanza", inplace=True)
ddrop=data.drop_duplicates(subset="event_id" )
ddrop=ddrop.drop("member_id", axis=1)
ddrop.sort_values("event_id")
ddrop=ddrop.reset_index()

In [5]:
d=data.groupby("event_id")
a=d['distanza'].quantile()
print(a[0])
ddrop['maxdist']=int(100000)
ddrop

7.0


,index,event_id,city,lat_member,lon_member,lat_event,lon_event,distanza,maxdist
0,369,260082067,Milano,45.459999,9.19,45.46,9.19,0.000107,100000
1,368,260139368,Milano,45.460000,9.20,45.46,9.19,0.780803,100000
2,362,260158872,Milano,45.470000,9.19,45.46,9.19,1.113195,100000
3,359,260398669,Milano,45.470000,9.19,45.46,9.19,1.113195,100000
4,360,259124091,Milano,45.470000,9.19,45.46,9.19,1.113195,100000
5,361,259530309,Milano,45.470000,9.19,45.46,9.19,1.113195,100000
6,364,260338784,Milano,45.470000,9.19,45.46,9.19,1.113195,100000
7,365,260306056,Milano,45.470000,9.19,45.46,9.19,1.113195,100000
8,366,qkmlrqyzgbxb,Milano,45.470000,9.19,45.46,9.19,1.113195,100000
9,367,tjbfrqyzgbfb,Milano,45.470000,9.19,45.46,9.19,1.113195,100000


In [6]:
for line in ddrop.itertuples():
    ddrop['maxdist'].at[line.Index]=int(a[line.Index])

type(ddrop['maxdist'][0])
#ddrop['maxdist']

numpy.int64

In [7]:
# I can add marker one by one on the map
ddrop=ddrop.sort_values("maxdist")
ddrop=ddrop.reset_index()

In [8]:
ddrop=ddrop.drop("level_0", axis=1)
ddrop=ddrop.drop("index", axis=1)


In [9]:
ddrop

,event_id,city,lat_member,lon_member,lat_event,lon_event,distanza,maxdist
0,ktsrvqyzgbmb,Milano,45.460000,9.19,45.46,9.19,7.000000,1
1,bxkhvqyzgbnb,Milano,45.630000,9.33,45.46,9.19,21.846313,1
2,260274350,Milano,45.390000,9.15,45.46,9.19,8.395687,1
3,sdgwsqyzgbhc,Milano,45.460000,9.19,45.46,9.19,7.000000,4
4,260398669,Milano,45.470000,9.19,45.46,9.19,1.113195,4
5,cqrgwqyzgbcc,Milano,45.460000,9.19,45.46,9.19,7.000000,5
6,260082067,Milano,45.459999,9.19,45.46,9.19,0.000107,7
7,259724285,Milano,45.460000,9.19,45.46,9.19,7.000000,7
8,259618744,Milano,45.460000,9.19,45.46,9.19,7.000000,7
9,259938608,Milano,45.460000,9.19,45.46,9.19,7.000000,7


In [10]:
# Make an empty map
m = folium.Map(location=[20,0], tiles="Mapbox Bright", zoom_start=2)

for line in ddrop.itertuples():
    print(data.iloc[line.Index]['event_id'])
    folium.Circle(
      location=[float(ddrop.iloc[line.Index]['lat_event']), float(ddrop.iloc[line.Index]['lon_event'])],
      popup=data.iloc[line.Index]['event_id'],
      radius=int(ddrop.iloc[line.Index]['maxdist'])*1000,
      #radius=1,
      color='crimson',
      fill=False,
      #fill_color='crimson'
    ).add_to(m)
    
#Save it as html
m.save('/home/dario/NeoMeetup/Visualizations/map/absmymap.html')

260082067
260139368
260158872
260398669
259124091
259530309
260158872
260338784
260306056
qkmlrqyzgbxb
tjbfrqyzgbfb
260407602
260258067
260314253
260262116
260395220
vpngwqyzgblc
whlfwqyzgbfc
cwjjvqyzgbsb
vpngwqyzgbsb
259827725
260548090
260306056
qkmlrqyzgbxb
260154185
260088430
260088430
260395340
260314253
260339255
260314253
260532363
260088430
260291412
260088430
260338075
260303890
260064491
260314253
260109079
sdgwsqyzgbpb
260338784
259699936
260242427
260078910
260242427
cwjjvqyzgbsb
260395340
260277040
260303890
259931890
260532744
sdgwsqyzgbpb
258842222
259530309
260303890
260338784
260096669
260096669
260096669
259892392
260096669
260538612
260240987
260240987
260240987
260240987
260240987
260240987
260240987
260240987
cqrgwqyzgbcc
260109079
260109079
260109079
260109079
260109079
260109079
260109079
260109079
260109079
260109079
260109079
260109079
260109079
260109079
260109079
260109079
260109079
260109079
260240987
260240987
260240987
260240987
260538612
260240987
2602409

In [11]:
#m1 = folium.Map(location=[20,0], tiles="Mapbox Bright", zoom_start=2)
from folium.plugins import MarkerCluster

map = folium.Map(location=[ddrop.iloc[1]['lat_event'],ddrop.iloc[1]['lon_event']], zoom_start=12)
mc=MarkerCluster()
for line in ddrop.itertuples():
    lat=ddrop.iloc[line.Index]['lat_event']+((line.Index))/1000
    lon=ddrop.iloc[line.Index]['lon_event']+((line.Index))/1000
    mc.add_child(
        folium.Marker(
        location = [lat, lon] 
        ))
map.add_child(mc)
display(map)
    
#Save it as html
map.save('/home/dario/NeoMeetup/Visualizations/map/try.html')

In [12]:
# Make an empty map
m2 = folium.Map(location=[20,0], tiles="OpenStreetMap", zoom_start=2)
mc=MarkerCluster( name='1000 clustered icons',
    overlay=True,
    control=True,
    icon_create_function=None)
for line in ddrop.itertuples():
    #print(data.iloc[line.Index]['event_id'])
    lat=ddrop.iloc[line.Index]['lat_event']#+((line.Index))/10000
    lon=ddrop.iloc[line.Index]['lon_event']#+((line.Index))/10000
    
    mc.add_child(folium.Circle(
      location=[lat, lon],
      popup=data.iloc[line.Index]['event_id'],
      radius=int(ddrop.iloc[line.Index]['maxdist'])*1000,
      #radius=1,
      color='crimson',
      fill=False,
      #fill_color='crimson'
    ))
    if line.Index is 3:
        break
m2.add_child(mc)
folium.LayerControl().add_to(m2)
display(m2)
#Save it as html
m2.save('/home/dario/NeoMeetup/Visualizations/map/mymap.html')

In [15]:
from folium.plugins import FastMarkerCluster
some_map = folium.Map(location=[ddrop['lat_event'].mean(), 
                                ddrop['lon_event'].mean()], 
                                zoom_start=4)

callback = ('function (row) {' 
                'var circle = L.circle(new L.LatLng(row[0], row[1]), {color: "red",  radius: row[2]*1000});'
                'return circle};')

some_map.add_child(FastMarkerCluster(ddrop[['lat_event', 'lon_event', 'maxdist', 'distanza']].values.tolist(), callback=callback))
some_map

In [14]:
ddrop[['lat_event', 'lon_event', 'maxdist', 'distanza']]

,lat_event,lon_event,maxdist,distanza
0,45.46,9.19,1,7.000000
1,45.46,9.19,1,21.846313
2,45.46,9.19,1,8.395687
3,45.46,9.19,4,7.000000
4,45.46,9.19,4,1.113195
5,45.46,9.19,5,7.000000
6,45.46,9.19,7,0.000107
7,45.46,9.19,7,7.000000
8,45.46,9.19,7,7.000000
9,45.46,9.19,7,7.000000
